# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 27 2023 10:20AM,258207,16,9212608,Sartorius Bioprocess Solutions,Released
1,Feb 27 2023 10:16AM,258206,20,9211512,"Exact-Rx, Inc.",Released
2,Feb 27 2023 10:16AM,258206,20,9212614,"Exact-Rx, Inc.",Released
3,Feb 27 2023 10:15AM,258202,15,30014234,"Alliance Pharma, Inc.",Released
4,Feb 27 2023 10:15AM,258202,15,30014233,"Alliance Pharma, Inc.",Released
5,Feb 27 2023 10:15AM,258202,15,30014235,"Alliance Pharma, Inc.",Released
6,Feb 27 2023 10:15AM,258202,15,30014236,"Alliance Pharma, Inc.",Released
7,Feb 27 2023 10:15AM,258202,15,30014237,"Alliance Pharma, Inc.",Released
8,Feb 27 2023 10:15AM,258202,15,30014238,"Alliance Pharma, Inc.",Released
9,Feb 27 2023 10:15AM,258202,15,30014341,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,258201,Released,1
29,258202,Released,114
30,258203,Released,1
31,258206,Released,2
32,258207,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258201,NaN,1.0
258202,NaN,114.0
258203,NaN,1.0
258206,NaN,2.0
258207,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258149,0.0,18.0
258153,1.0,30.0
258157,1.0,0.0
258162,0.0,58.0
258166,9.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258149,0,18
258153,1,30
258157,1,0
258162,0,58
258166,9,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258149,0,18
1,258153,1,30
2,258157,1,0
3,258162,0,58
4,258166,9,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258149,,18
1,258153,1,30
2,258157,1,
3,258162,,58
4,258166,9,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 27 2023 10:20AM,258207,16,Sartorius Bioprocess Solutions
1,Feb 27 2023 10:16AM,258206,20,"Exact-Rx, Inc."
3,Feb 27 2023 10:15AM,258202,15,"Alliance Pharma, Inc."
117,Feb 27 2023 10:14AM,258199,10,SugarBear
118,Feb 27 2023 10:07AM,258203,19,Purify Life
119,Feb 27 2023 10:05AM,258201,16,Sartorius Bioprocess Solutions
120,Feb 27 2023 10:04AM,258200,16,Sartorius Bioprocess Solutions
121,Feb 27 2023 9:57AM,258198,16,Sartorius Lab Products and Service
125,Feb 27 2023 9:49AM,258197,10,"Citieffe, Inc."
130,Feb 27 2023 9:44AM,258196,20,"HVL, LLC dba Atrium Innovations"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 27 2023 10:20AM,258207,16,Sartorius Bioprocess Solutions,,1
1,Feb 27 2023 10:16AM,258206,20,"Exact-Rx, Inc.",,2
2,Feb 27 2023 10:15AM,258202,15,"Alliance Pharma, Inc.",,114
3,Feb 27 2023 10:14AM,258199,10,SugarBear,,1
4,Feb 27 2023 10:07AM,258203,19,Purify Life,,1
5,Feb 27 2023 10:05AM,258201,16,Sartorius Bioprocess Solutions,,1
6,Feb 27 2023 10:04AM,258200,16,Sartorius Bioprocess Solutions,,1
7,Feb 27 2023 9:57AM,258198,16,Sartorius Lab Products and Service,,4
8,Feb 27 2023 9:49AM,258197,10,"Citieffe, Inc.",2,3
9,Feb 27 2023 9:44AM,258196,20,"HVL, LLC dba Atrium Innovations",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 10:20AM,258207,16,Sartorius Bioprocess Solutions,1,
1,Feb 27 2023 10:16AM,258206,20,"Exact-Rx, Inc.",2,
2,Feb 27 2023 10:15AM,258202,15,"Alliance Pharma, Inc.",114,
3,Feb 27 2023 10:14AM,258199,10,SugarBear,1,
4,Feb 27 2023 10:07AM,258203,19,Purify Life,1,
5,Feb 27 2023 10:05AM,258201,16,Sartorius Bioprocess Solutions,1,
6,Feb 27 2023 10:04AM,258200,16,Sartorius Bioprocess Solutions,1,
7,Feb 27 2023 9:57AM,258198,16,Sartorius Lab Products and Service,4,
8,Feb 27 2023 9:49AM,258197,10,"Citieffe, Inc.",3,2
9,Feb 27 2023 9:44AM,258196,20,"HVL, LLC dba Atrium Innovations",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 10:20AM,258207,16,Sartorius Bioprocess Solutions,1,
1,Feb 27 2023 10:16AM,258206,20,"Exact-Rx, Inc.",2,
2,Feb 27 2023 10:15AM,258202,15,"Alliance Pharma, Inc.",114,
3,Feb 27 2023 10:14AM,258199,10,SugarBear,1,
4,Feb 27 2023 10:07AM,258203,19,Purify Life,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 10:20AM,258207,16,Sartorius Bioprocess Solutions,1.0,NaN
1,Feb 27 2023 10:16AM,258206,20,"Exact-Rx, Inc.",2.0,NaN
2,Feb 27 2023 10:15AM,258202,15,"Alliance Pharma, Inc.",114.0,NaN
3,Feb 27 2023 10:14AM,258199,10,SugarBear,1.0,NaN
4,Feb 27 2023 10:07AM,258203,19,Purify Life,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 10:20AM,258207,16,Sartorius Bioprocess Solutions,1.0,0.0
1,Feb 27 2023 10:16AM,258206,20,"Exact-Rx, Inc.",2.0,0.0
2,Feb 27 2023 10:15AM,258202,15,"Alliance Pharma, Inc.",114.0,0.0
3,Feb 27 2023 10:14AM,258199,10,SugarBear,1.0,0.0
4,Feb 27 2023 10:07AM,258203,19,Purify Life,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3614530,237.0,35.0
12,516369,54.0,0.0
15,258202,114.0,0.0
16,1032806,7.0,0.0
18,1032691,4.0,0.0
19,516369,10.0,9.0
20,516402,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3614530,237.0,35.0
1,12,516369,54.0,0.0
2,15,258202,114.0,0.0
3,16,1032806,7.0,0.0
4,18,1032691,4.0,0.0
5,19,516369,10.0,9.0
6,20,516402,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,237.0,35.0
1,12,54.0,0.0
2,15,114.0,0.0
3,16,7.0,0.0
4,18,4.0,0.0
5,19,10.0,9.0
6,20,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,237.0
1,12,Released,54.0
2,15,Released,114.0
3,16,Released,7.0
4,18,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Executing,35.0,0.0,0.0,0.0,0.0,9.0,0.0
Released,237.0,54.0,114.0,7.0,4.0,10.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Executing,35.0,0.0,0.0,0.0,0.0,9.0,0.0
1,Released,237.0,54.0,114.0,7.0,4.0,10.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Executing,35.0,0.0,0.0,0.0,0.0,9.0,0.0
1,Released,237.0,54.0,114.0,7.0,4.0,10.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()